In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
egfr_out = pd.read_csv('../data/crc_egfr_out.csv', index_col=0)
folfox_out = pd.read_csv('../data/crc_folfox_out.csv', index_col=0)
folfiri_out = pd.read_csv('../data/crc_folfiri_out.csv', index_col=0)
ib_out = pd.read_csv('../data/crc_ib_out.csv', index_col=0)
tri_out = pd.read_csv('../data/crc_tri_out.csv', index_col=0)

egfr_out.rename(columns={'os_g_status': 'OS', 'pfs_m_g_status': 'PFS'}, inplace=True)
folfox_out.rename(columns={'os_g_status': 'OS', 'pfs_m_g_status': 'PFS'}, inplace=True)
folfiri_out.rename(columns={'os_g_status': 'OS', 'pfs_m_g_status': 'PFS'}, inplace=True)
ib_out.rename(columns={'os_g_status': 'OS', 'pfs_m_g_status': 'PFS'}, inplace=True)
tri_out.rename(columns={'os_g_status': 'OS', 'pfs_m_g_status': 'PFS'}, inplace=True)

#add prefix to egfr_out columns except PFS and OS
egfr_out.columns = ['id_' + i if i not in ['PFS', 'OS'] else i for i in egfr_out.columns]
folfox_out.columns = ['id_' + i if i not in ['PFS', 'OS'] else i for i in folfox_out.columns]
folfiri_out.columns = ['id_' + i if i not in ['PFS', 'OS'] else i for i in folfiri_out.columns]
ib_out.columns = ['id_' + i if i not in ['PFS', 'OS'] else i for i in ib_out.columns]
tri_out.columns = ['id_' + i if i not in ['PFS', 'OS'] else i for i in tri_out.columns]
egfr_out.head()

,id_record_id,id_institution,id_drugs_list,OS,id_tt_os_g_mos,PFS,id_tt_pfs_m_g_mos,id_sample_id
0,GENIE-DFCI-000971,DFCI,"Bevacizumab, Cetuximab, Irinotecan Hydrochloride",1,11.546053,1.0,5.723684,GENIE-DFCI-000971-10958
1,GENIE-DFCI-001292,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin,...",1,10.493421,1.0,4.144737,GENIE-DFCI-001292-7406
2,GENIE-DFCI-001463,DFCI,Cetuximab,1,9.703947,0.0,5.756579,GENIE-DFCI-001463-9650
3,GENIE-DFCI-001463,DFCI,"Cetuximab, Irinotecan Hydrochloride",1,3.947368,1.0,2.927632,GENIE-DFCI-001463-9650
4,GENIE-DFCI-002507,DFCI,"Fluorouracil, Irinotecan Hydrochloride, Leucov...",0,35.427632,1.0,5.953947,GENIE-DFCI-002507-4815


In [4]:
mut = pd.read_csv('../data/crc_mutoh_pertreat.csv', index_col=0)
mut.columns = ['mut_' + i for i in mut.columns]
cna = pd.read_csv('../data/crc_cna_pertreat.csv', index_col=0)
cna.columns = ['cna_' + i for i in cna.columns]
fusion = pd.read_csv('../data/crc_fusion.csv', index_col=0)
fusion.columns = ['fus_' + i for i in fusion.columns]
clin = pd.read_csv('../data/crc_clin_pub.csv', index_col=0)
#add prefix to clin columns except PFS and OS
clin.columns = ['clin_' + i if i not in ['PFS', 'OS', 'OS_time'] else i for i in clin.columns]
#drop 'OS' and 'PFS' columns
clin = clin.drop(['OS', 'PFS'], axis=1)
cat_cols = [col for col in clin.columns if col.startswith('clin') and clin[col].nunique() <= 10]
num_cols = [col for col in clin.columns if col.startswith('clin') and clin[col].nunique() > 10]
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in cat_cols:
    clin[i] = le.fit_transform(clin[i])
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy='median')
for i in num_cols:
    clin[i] = imp.fit_transform(clin[i].values.reshape(-1,1))

In [5]:
mut.head()

,mut_CDK4,mut_CCND3,mut_CDH1,mut_CDK8,mut_GNAS,mut_PRKAR1A,mut_MLH1,mut_PNRC1,mut_ETV1,mut_TSC1,...,mut_ROS1,mut_DAXX,mut_TSC2,mut_RAD51C,mut_KMT2D,mut_ARID1A,mut_AURKB,mut_U2AF1,mut_REL,mut_BRCA1
GENIE-DFCI-002643-6598,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
GENIE-DFCI-008624-7126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GENIE-DFCI-003423-6929,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GENIE-DFCI-007675-6942,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
GENIE-DFCI-008889-6970,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
cna.head()

,cna_ABL1,cna_AKT1,cna_AKT2,cna_AKT3,cna_ALK,cna_ALOX12B,cna_APC,cna_AR,cna_ARAF,cna_ARID1A,...,cna_TMPRSS2,cna_TNFAIP3,cna_TP53,cna_TSC1,cna_TSC2,cna_TSHR,cna_U2AF1,cna_VHL,cna_WT1,cna_XPO1
sample_id,,,,,,,,,,,,,,,,,,,,,
GENIE-DFCI-002643-6598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GENIE-DFCI-008624-7126,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GENIE-DFCI-003423-6929,0.0,-1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,...,-1.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
GENIE-DFCI-007675-6942,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
GENIE-DFCI-008889-6970,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
egfr = egfr_out.set_index('id_sample_id')
ib = ib_out.set_index('id_sample_id')
folfox = folfox_out.set_index('id_sample_id')
folfiri = folfiri_out.set_index('id_sample_id')

tri = tri_out.set_index('id_sample_id')
egfr_mut = egfr.join(mut, how='inner')
egfr_mut_cna = egfr_mut.join(cna, how='inner')
egfr_mut_cna_fus = egfr_mut_cna.join(fusion, how='inner')
print(egfr_mut_cna.shape)
folfiri_mut = folfiri.join(mut, how='inner')
folfiri_mut_cna = folfiri_mut.join(cna, how='inner')
folfiri_mut_cna_fus = folfiri_mut_cna.join(fusion, how='inner')
print(folfiri_mut_cna.shape)
tri_mut = tri.join(mut, how='inner')
tri_mut_cna = tri_mut.join(cna, how='inner')
tri_mut_cna_fus = tri_mut_cna.join(fusion, how='inner')
print(tri_mut_cna.shape)

ib_mut = ib.join(mut, how='inner')
ib_mut_cna = ib_mut.join(cna, how='inner')
ib_mut_cna_fus = ib_mut_cna.join(fusion, how='inner')
print(ib_mut_cna.shape)

folfox_mut = folfox.join(mut, how='inner')
folfox_mut_cna = folfox_mut.join(cna, how='inner')
folfox_mut_cna_fus = folfox_mut_cna.join(fusion, how='inner')
print(folfox_mut_cna.shape)

(354, 455)
(909, 455)
(148, 455)
(146, 455)
(1290, 455)


In [8]:
egfr_mut_cna_clin_fus = egfr_mut_cna.set_index('id_record_id').join(clin, how='inner')
folfiri_mut_cna_clin_fus = folfiri_mut_cna.set_index('id_record_id').join(clin, how='inner')
tri_mut_cna_clin_fus = tri_mut_cna.set_index('id_record_id').join(clin, how='inner')
ib_mut_cna_clin_fus = ib_mut_cna.set_index('id_record_id').join(clin, how='inner')
folfox_mut_cna_clin_fus = folfox_mut_cna.set_index('id_record_id').join(clin, how='inner')
print(egfr_mut_cna_clin_fus.shape, ib_mut_cna_clin_fus.shape, folfox_mut_cna_clin_fus.shape, folfiri_mut_cna_clin_fus.shape, tri_mut_cna_clin_fus.shape)

(354, 476) (146, 476) (1290, 476) (909, 476) (148, 476)


In [9]:
egfr_mut_cna_clin_fus.head()

,id_institution,id_drugs_list,OS,id_tt_os_g_mos,PFS,id_tt_pfs_m_g_mos,mut_CDK4,mut_CCND3,mut_CDH1,mut_CDK8,...,clin_ca_first_dmets1,clin_ca_crc_td,clin_ca_crc_crm,clin_ca_crc_peri_inv,clin_crc_type,OS_time,clin_Histology Category,clin_Histology,clin_Derived Grade or Differentiation of Tumor,clin_CEA
GENIE-DFCI-000971,DFCI,"Bevacizumab, Cetuximab, Irinotecan Hydrochloride",1,11.546053,1.0,5.723684,0,0,0,0,...,0,1,0,0,1,814,0,0,4,9.3
GENIE-DFCI-001292,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin,...",1,10.493421,1.0,4.144737,0,0,0,0,...,5,2,3,0,2,637,0,0,4,20.0
GENIE-DFCI-001463,DFCI,Cetuximab,1,9.703947,0.0,5.756579,0,0,0,0,...,5,2,2,0,0,5468,0,0,1,0.5
GENIE-DFCI-001463,DFCI,"Cetuximab, Irinotecan Hydrochloride",1,3.947368,1.0,2.927632,0,0,0,0,...,5,2,2,0,0,5468,0,0,1,0.5
GENIE-DFCI-002507,DFCI,"Fluorouracil, Irinotecan Hydrochloride, Leucov...",0,35.427632,1.0,5.953947,0,0,0,0,...,0,2,2,0,0,2288,0,0,3,83.4


In [10]:
ib_mut_cna_clin_fus.head()

,id_institution,id_drugs_list,OS,PFS,id_tt_os_g_mos,id_tt_pfs_m_g_mos,mut_CDK4,mut_CCND3,mut_CDH1,mut_CDK8,...,clin_ca_first_dmets1,clin_ca_crc_td,clin_ca_crc_crm,clin_ca_crc_peri_inv,clin_crc_type,OS_time,clin_Histology Category,clin_Histology,clin_Derived Grade or Differentiation of Tumor,clin_CEA
GENIE-DFCI-000971,DFCI,Regorafenib,1,1.0,0.625000,0.361842,0,0,0,0,...,0,1,0,0,1,814,0,0,4,9.3
GENIE-DFCI-001038,DFCI,Regorafenib,0,1.0,23.223684,3.157895,0,0,0,0,...,0,2,0,0,0,1465,0,0,4,100.0
GENIE-DFCI-002499,DFCI,Cabozantinib Smalate,1,0.0,8.881579,0.921053,0,0,0,0,...,1,1,1,1,0,1343,0,0,1,100.0
GENIE-DFCI-002507,DFCI,Regorafenib,1,1.0,4.802632,2.697368,0,0,0,0,...,0,2,2,0,0,2288,0,0,3,83.4
GENIE-DFCI-002561,DFCI,Regorafenib,1,1.0,10.789474,4.605263,0,0,0,0,...,0,1,2,1,0,2140,0,0,1,8.4


In [11]:
folfox_mut_cna_clin_fus.head()

,id_institution,id_drugs_list,OS,id_tt_os_g_mos,PFS,id_tt_pfs_m_g_mos,mut_CDK4,mut_CCND3,mut_CDH1,mut_CDK8,...,clin_ca_first_dmets1,clin_ca_crc_td,clin_ca_crc_crm,clin_ca_crc_peri_inv,clin_crc_type,OS_time,clin_Histology Category,clin_Histology,clin_Derived Grade or Differentiation of Tumor,clin_CEA
GENIE-DFCI-000233,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",0,100.986842,0.0,31.578947,0,0,0,0,...,5,2,2,0,2,3303,0,0,1,1.0
GENIE-DFCI-000247,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",1,35.789474,0.0,3.914474,0,0,0,0,...,0,2,0,0,2,1163,0,0,0,4.9
GENIE-DFCI-000306,DFCI,"bev, Fluorouracil, Leucovorin Calcium, Oxalipl...",0,38.125000,0.0,12.960526,0,0,0,0,...,1,2,2,0,3,1230,0,0,1,0.7
GENIE-DFCI-000738,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",0,51.578947,NaN,NaN,0,0,0,0,...,5,0,2,0,3,1623,0,1,1,1.8
GENIE-DFCI-000924,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",0,65.361842,NaN,NaN,0,0,0,0,...,5,0,2,1,0,2039,0,0,1,0.6


In [12]:
egfr_mut_cna_clin_fus.to_csv('../data/egfr_mut_cna_fus_clin.csv')
folfiri_mut_cna_clin_fus.to_csv('../data/folfiri_mut_cna_fus_clin.csv')
tri_mut_cna_clin_fus.to_csv('../data/tri_mut_cna_fus_clin.csv')
ib_mut_cna_clin_fus.to_csv('../data/ib_mut_cna_fus_clin.csv')
folfox_mut_cna_clin_fus.to_csv('../data/folfox_mut_cna_fus_clin.csv')